In [127]:
from IPython.core.display import HTML
HTML("""
<style>
    h1{background-color:black; color:white; padding: 10px 10px 10px 10px}
    h2{background-color:green; color:white;  padding: 5px 5px 5px 5px}
</style>
""")

In [124]:
import pandas as pd
import numpy as np
import json
import datetime
import itertools
import re

# Config Parameters

In [36]:
## Data on Exchanges and Returns
FILEPATH_DATA_EXCHANGES_RETURNS = '../data/data_science_exchanges_returns.file'

## Data Conversions
FILEPATH_DATA_CONVERSIONS = '../data/data_science_test_conversions.file'

## Period Range for analysis
ORDER_START_DATE = datetime.date(2019,2,1)
ORDER_END_DATE = datetime.date(2019,2,28)

# Utility Functions

In [37]:
## Function to normalize character string
## - replace NaN with '<NA>'
## - convert to lowercase
## - remove white space
def normalize_string(string):
    return '<NA>' if type(string) is not str else ''.join(string.lower().strip().split())

# Data on Exchanges / Returns

## Read in dataset

In [38]:
DATA_EXCHANGE_RETURNS = pd.read_table(
    FILEPATH_DATA_EXCHANGES_RETURNS,
    delimiter='\t',
    dtype = {'Order nr':'object'}
)

/home/winston/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (3,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
DATA_EXCHANGE_RETURNS.head()

,Country,Order nr,SKU ID,Reason Name,original_size_system_name,original_size_name,exchanged_size_system_name,exchanged_size_name
0,ph,239748222,EV032US96WVNPH,1._size_does_not_fit_-_too_large,International,One Size,NaN,NaN
1,ph,251645722,CH672SH32LPNPH,2._size_does_not_fit_-_too_small,US,7,NaN,NaN
2,ph,235646762,EM520AC06VDDPH,1._size_does_not_fit_-_too_large,International,One Size,NaN,NaN
3,ph,275962462,RU506SH93IHUMY,1._size_does_not_fit_-_too_large,EU,36,NaN,NaN
4,ph,259521762,MC141AC39HVKPH,2._size_does_not_fit_-_too_small,US,M,NaN,NaN


In [40]:
DATA_EXCHANGE_RETURNS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387055 entries, 0 to 387054
Data columns (total 8 columns):
Country                       387055 non-null object
Order nr                      386344 non-null object
SKU ID                        387055 non-null object
Reason Name                   91242 non-null object
original_size_system_name     385998 non-null object
original_size_name            386344 non-null object
exchanged_size_system_name    295813 non-null object
exchanged_size_name           295813 non-null object
dtypes: object(8)
memory usage: 23.6+ MB


## Filter for 'size_does_not_fit' reasons

In [41]:
DATA_EXCHANGE_RETURNS['Reason Name'].unique()

array(['1._size_does_not_fit_-_too_large',
       '2._size_does_not_fit_-_too_small', nan], dtype=object)

In [42]:
selected_reasons = {
    '1._size_does_not_fit_-_too_large':'too_large',
    '2._size_does_not_fit_-_too_small':'too_small'
}

In [43]:
DATA_EXCHANGE_RETURNS = DATA_EXCHANGE_RETURNS[
    DATA_EXCHANGE_RETURNS['Reason Name'].map(lambda x: x in list(selected_reasons.keys()))
]

In [44]:
DATA_EXCHANGE_RETURNS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91242 entries, 0 to 98282
Data columns (total 8 columns):
Country                       91242 non-null object
Order nr                      91242 non-null object
SKU ID                        91242 non-null object
Reason Name                   91242 non-null object
original_size_system_name     90896 non-null object
original_size_name            91242 non-null object
exchanged_size_system_name    0 non-null object
exchanged_size_name           0 non-null object
dtypes: object(8)
memory usage: 6.3+ MB


## Simplify Reason Names

In [45]:
DATA_EXCHANGE_RETURNS['Reason Name'] = DATA_EXCHANGE_RETURNS['Reason Name'].map(selected_reasons)

## Normalize strings for size system names and size names

In [46]:
for sizetype in ['original','exchanged']:
    for sizenametype in ['size_system_name','size_name']:
        DATA_EXCHANGE_RETURNS[f'{sizetype}_{sizenametype}'] = \
          DATA_EXCHANGE_RETURNS[f'{sizetype}_{sizenametype}'].apply(normalize_string)

## Get original and exchanged sizes by joining size system name with size name

In [47]:
for sizetype in ['original','exchanged']:
    DATA_EXCHANGE_RETURNS[f'{sizetype}_size'] = [
        str(sys_name) + ' ' + str(name) 
        for sys_name,name in zip(
            DATA_EXCHANGE_RETURNS[f'{sizetype}_size_system_name'],
            DATA_EXCHANGE_RETURNS[f'{sizetype}_size_name']
        )
    ]
    
    DATA_EXCHANGE_RETURNS[f'{sizetype}_size'] = DATA_EXCHANGE_RETURNS[f'{sizetype}_size'].apply(normalize_string)

In [50]:
DATA_EXCHANGE_RETURNS.shape

(91242, 10)

In [48]:
DATA_EXCHANGE_RETURNS.head()

,Country,Order nr,SKU ID,Reason Name,original_size_system_name,original_size_name,exchanged_size_system_name,exchanged_size_name,original_size,exchanged_size
0,ph,239748222,EV032US96WVNPH,too_large,international,onesize,<NA>,<NA>,internationalonesize,<na><na>
1,ph,251645722,CH672SH32LPNPH,too_small,us,7,<NA>,<NA>,us7,<na><na>
2,ph,235646762,EM520AC06VDDPH,too_large,international,onesize,<NA>,<NA>,internationalonesize,<na><na>
3,ph,275962462,RU506SH93IHUMY,too_large,eu,36,<NA>,<NA>,eu36,<na><na>
4,ph,259521762,MC141AC39HVKPH,too_small,us,m,<NA>,<NA>,usm,<na><na>


## Columns of interest

In [51]:
DATA_EXCHANGE_RETURNS = DATA_EXCHANGE_RETURNS[[
    'Order nr',
    'SKU ID',
    'Reason Name',
    'original_size',
    'exchanged_size'
]]

## Consolidating unique records

In [54]:
DATA_EXCHANGE_RETURNS.head(10)

,Order nr,SKU ID,Reason Name,original_size,exchanged_size
0,239748222,EV032US96WVNPH,too_large,internationalonesize,<na><na>
1,251645722,CH672SH32LPNPH,too_small,us7,<na><na>
2,235646762,EM520AC06VDDPH,too_large,internationalonesize,<na><na>
3,275962462,RU506SH93IHUMY,too_large,eu36,<na><na>
4,259521762,MC141AC39HVKPH,too_small,usm,<na><na>
5,231545122,SP132SH0IJXLPH,too_small,us9,<na><na>
6,228331822,CH672SH99QBCPH,too_small,us8,<na><na>
7,276191162,LO601AA67KQUPH,too_small,internationalxl,<na><na>
8,282982362,F1525AA74BFHPH,too_large,internationalonesize,<na><na>
9,282982362,F1525AA74BFHPH,too_large,internationalonesize,<na><na>


Each exchange/return record shall be uniquely identified by

* **Transaction ID** -- *Order nr*
* **Product ID purchased in that Transaction** -- *SKU ID*
* **Size of Product Purchased for that Product ID** -- *original_size*

### Remove duplicate records

In [65]:
print('{0} out of {1} rows are duplicate records ({2}%)'.format(
    DATA_EXCHANGE_RETURNS.duplicated(keep='first').sum(),
    DATA_EXCHANGE_RETURNS.shape[0],
    round(100*DATA_EXCHANGE_RETURNS.duplicated(keep='first').sum()/DATA_EXCHANGE_RETURNS.shape[0],2)
))

467 out of 91242 rows are duplicate records (0.51%)


In [66]:
DATA_EXCHANGE_RETURNS = DATA_EXCHANGE_RETURNS[~DATA_EXCHANGE_RETURNS.duplicated(keep='first')]

In [68]:
DATA_EXCHANGE_RETURNS.shape

(90775, 5)

### For each unique exchange/return record, concatenate all the associated **Reason Names** and **Exchanged Sizes** if there are more than 1 

In [69]:
DATA_EXCHANGE_RETURNS = DATA_EXCHANGE_RETURNS\
  .groupby(['Order nr','SKU ID','original_size'])\
  .agg({
    'Reason Name' : lambda x: '__AND__'.join(sorted(list(set(x)))),
    'exchanged_size' : lambda x: '__AND__'.join(sorted(list(set(x))))
  })\
  .reset_index()

In [71]:
DATA_EXCHANGE_RETURNS.shape

(90740, 5)

In [72]:
DATA_EXCHANGE_RETURNS.head()

,Order nr,SKU ID,original_size,Reason Name,exchanged_size
0,211113522,CBF67AA8DA5D5EGS,internationals,too_large,<na><na>
1,211113662,AE351AA0039298GS,internationall,too_small,<na><na>
2,211113822,BI090SH55HNOMY,eu39,too_large,<na><na>
3,211114862,C8695AAD6F487CGS,eu38,too_small,<na><na>
4,211114922,5E1B9AA609044BGS,internationalm,too_large,<na><na>


In [83]:
## 35 records with more than 1 Reason Names (includes both too large and too small)
DATA_EXCHANGE_RETURNS[DATA_EXCHANGE_RETURNS['Reason Name'].apply(
    lambda x: True if re.search(pattern='_AND_',string=x) else False
)].shape

(35, 5)

## Summary of data processing steps

1. Filter for 'size_does_not_fit' related reason names
  * Original row count : 387055 ==> New row count : 91242


2. Get original and exchanged sizes (system_name concat with size_name)


3. Normalize original / exchanged size strings
  * strip whitespace
  * lowercasing


4. Removing duplicates
  * Original row count : 91242 ==> New row count : 90775
  
  
5. Consolidate Reason Names and Exchanged sizes for each unique exchange/return record
  * Original row count : 90775 ==> New row count : 90740

# Data on Conversions

## Read in Conversions data (data dump from json stream, 1 line per record)

In [128]:
JSON_DUMP = []

In [129]:
with open(FILEPATH_DATA_CONVERSIONS) as f:
    for line in f.readlines():
        JSON_DUMP.append(json.loads(line))

In [130]:
len(JSON_DUMP)

62482

In [131]:
JSON_DUMP[:1]

[{'_index': 'tracking_prod',
  '_type': 'conversion',
  '_id': '8a8b6404-e312-4bef-98cd-df3637d09b2f',
  '_score': 7.5778923,
  '_source': {'uid': 'unknown',
   'amount': '1394',
   'clientId': 'ubu8zhxrh8hg',
   'currency': 'PHP',
   'transaction': '257871622',
   'products': [{'quantity': 1,
     'size': 'International L',
     'price': '413.59',
     'sku': '3AE72AA36B07C6GS'},
    {'quantity': 1,
     'size': 'International M',
     'price': '465.36',
     'sku': 'B9586AA24B4E70GS'},
    {'quantity': 1,
     'size': 'International M',
     'price': '515.05',
     'sku': '079A3AAEAB518DGS'}],
   'timestamp': '2018-11-09T05:59:08.254Z'}}]

The data we are interested in is nested inside the **_source** key in the record

In [132]:
DATA_CONVERSIONS = pd.DataFrame.from_records([
    record['_source'] for record in JSON_DUMP
])

In [133]:
DATA_CONVERSIONS.head()

,amount,clientId,currency,products,timestamp,transaction,uid
0,1394,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International L', 'p...",2018-11-09T05:59:08.254Z,257871622,unknown
1,299,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International One Si...",2018-11-09T06:32:29.749Z,265371622,0fc636a4-a03f-493d-8122-64f98c12e6df
2,24000,ubu8zhxrh8hg,PHP,"[{'quantity': 12, 'size': 'International One S...",2018-11-09T08:40:44.673Z,231351622,3c3c50b0-99a2-494e-8228-b559eafccc02
3,1299,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'Brazil 37/38', 'pric...",2018-11-09T11:52:16.426Z,294431622,unknown
4,2091.56,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International L', 'p...",2018-11-09T12:55:39.748Z,218931622,unknown


## Parse Date Time

In [134]:
DATA_CONVERSIONS['timestamp_parsed'] = DATA_CONVERSIONS['timestamp'].map(
    lambda x: datetime.datetime.strptime(x,'%Y-%m-%dT%H:%M:%S.%fz')
)

## Filter for transactions falling within date range of interest

In [135]:
DATA_CONVERSIONS = DATA_CONVERSIONS[DATA_CONVERSIONS['timestamp_parsed'].map(lambda x: x.date() >= ORDER_START_DATE)]
DATA_CONVERSIONS = DATA_CONVERSIONS[DATA_CONVERSIONS['timestamp_parsed'].map(lambda x: x.date() <= ORDER_END_DATE)]

In [136]:
DATA_CONVERSIONS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4705 entries, 5185 to 56886
Data columns (total 8 columns):
amount              4705 non-null object
clientId            4705 non-null object
currency            4705 non-null object
products            4705 non-null object
timestamp           4705 non-null object
transaction         4705 non-null object
uid                 4705 non-null object
timestamp_parsed    4705 non-null datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 330.8+ KB


In [137]:
print(np.min(DATA_CONVERSIONS['timestamp_parsed']))
print(np.max(DATA_CONVERSIONS['timestamp_parsed']))

2019-02-01 00:03:44.419000
2019-02-28 23:51:17.679000


In [138]:
DATA_CONVERSIONS.head()

,amount,clientId,currency,products,timestamp,transaction,uid,timestamp_parsed
5185,1143.12,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International S', 'p...",2019-02-07T08:29:55.536Z,215125962,unknown,2019-02-07 08:29:55.536
5186,1150,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'UK 10', 'price': '11...",2019-02-07T08:49:49.141Z,215325962,f6fb8346-1b77-4e7f-a631-cfb4b571ceb6,2019-02-07 08:49:49.141
5187,1996,ubu8zhxrh8hg,PHP,"[{'quantity': 2, 'size': 'International XXXL',...",2019-02-07T10:04:50.854Z,279965962,c5c23898-6998-4ae3-b4e9-9f4477ecd6bf,2019-02-07 10:04:50.854
5188,3650,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International One Si...",2019-02-07T17:41:34.990Z,267385962,a695c21f-e7ea-4a80-8719-a658a9af045e,2019-02-07 17:41:34.990
5189,1750,ubu8zhxrh8hg,PHP,"[{'quantity': 1, 'size': 'International One Si...",2019-02-08T01:31:22.243Z,273455962,unknown,2019-02-08 01:31:22.243


## Extract Products in Transactions

In [143]:
# Number of unique transaction IDs
len(DATA_CONVERSIONS.transaction.unique())

4705

In [144]:
# Number of rows
DATA_CONVERSIONS.shape[0]

4705

Each data conversion record corresponds to 1 unique transaction ID

In [147]:
DATA_CONVERSION_PRODUCTS = DATA_CONVERSIONS['products'].tolist()

In [148]:
# For each product object in each transaction record, append the transaction id
# This is to facilitate conversion to dataframe later on
for i in range(DATA_CONVERSIONS.shape[0]):
    for obj in DATA_CONVERSION_PRODUCTS[i]:
        obj['transaction'] = DATA_CONVERSIONS['transaction'].tolist()[i]

In [149]:
# flatten / unnest list of records
DATA_CONVERSION_PRODUCTS = list(itertools.chain.from_iterable(DATA_CONVERSION_PRODUCTS))

In [150]:
# format data into pandas dataframe
DATA_CONVERSION_PRODUCTS = pd.DataFrame.from_records(DATA_CONVERSION_PRODUCTS)

In [151]:
# remove records where sku is undefined
DATA_CONVERSION_PRODUCTS = DATA_CONVERSION_PRODUCTS[~(DATA_CONVERSION_PRODUCTS['sku']=='undefined')]

In [152]:
DATA_CONVERSION_PRODUCTS.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8578 entries, 0 to 8599
Data columns (total 5 columns):
price          8578 non-null object
quantity       8578 non-null float64
size           8578 non-null object
sku            8578 non-null object
transaction    8578 non-null object
dtypes: float64(1), object(4)
memory usage: 402.1+ KB


In [191]:
DATA_CONVERSION_PRODUCTS.head()

,price,quantity,size,sku,transaction,size_name
0,1143.12,1.0,International S,04BF2AA2B31011GS,215125962,internationals
1,1150,1.0,UK 10,79CE8US7578ECEGS,215325962,uk10
2,998,2.0,International XXXL,6C34EAAE0207FDGS,279965962,internationalxxxl
3,3650,1.0,International One Size,4524FAC279C76EGS,267385962,internationalonesize
4,1750,1.0,International One Size,DB581ACD3B11F1GS,273455962,internationalonesize


## Check for duplicates

In [154]:
DATA_CONVERSION_PRODUCTS.duplicated().sum() ## No duplicates

0

## Standardize size names

In [155]:
DATA_CONVERSION_PRODUCTS['size_name'] = DATA_CONVERSION_PRODUCTS['size'].apply(normalize_string)

In [156]:
DATA_CONVERSION_PRODUCTS.head()

,price,quantity,size,sku,transaction,size_name
0,1143.12,1.0,International S,04BF2AA2B31011GS,215125962,internationals
1,1150,1.0,UK 10,79CE8US7578ECEGS,215325962,uk10
2,998,2.0,International XXXL,6C34EAAE0207FDGS,279965962,internationalxxxl
3,3650,1.0,International One Size,4524FAC279C76EGS,267385962,internationalonesize
4,1750,1.0,International One Size,DB581ACD3B11F1GS,273455962,internationalonesize


# Merge Exchange/Return Data with Conversion Data

## Step 1 : Find SKU IDs of Transaction IDs that were returned, regardless of size

In [157]:
DATA_CONVERSIONS_EXCHANGE_RETURNS = DATA_CONVERSION_PRODUCTS.merge(
  right = DATA_EXCHANGE_RETURNS[['Order nr','SKU ID']].drop_duplicates(),
  how = 'left',
  left_on = ['transaction','sku'],
  right_on = ['Order nr','SKU ID'],
  indicator = True
)

In [158]:
# ind_exchangereturn is an indicator variable of whether the product (SKUID) in the transaction (transaction)
# was exchanged/returned with a stated size related reason (too_large/too_small)
DATA_CONVERSIONS_EXCHANGE_RETURNS = DATA_CONVERSIONS_EXCHANGE_RETURNS.rename(
    columns={'_merge':'ind_exchangereturn'}
)

In [159]:
DATA_CONVERSIONS_EXCHANGE_RETURNS['ind_exchangereturn'] = \
    DATA_CONVERSIONS_EXCHANGE_RETURNS['ind_exchangereturn'].map({
        'both':1,
        'left_only':0
    })

In [160]:
DATA_CONVERSIONS_EXCHANGE_RETURNS.sort_values('ind_exchangereturn').tail()

,price,quantity,size,sku,transaction,size_name,Order nr,SKU ID,ind_exchangereturn
8208,831.2,1.0,International XS,68F75AA80E37FCGS,221563862,internationalxs,221563862,68F75AA80E37FCGS,1
5979,704.25,1.0,International XL,4ABCEAAD9A3FDBGS,262794862,internationalxl,262794862,4ABCEAAD9A3FDBGS,1
5978,974.25,1.0,International XL,C7503AA6E8554CGS,262794862,internationalxl,262794862,C7503AA6E8554CGS,1
8222,1295,1.0,International XL,E9468AA1B4AEB1GS,244553862,internationalxl,244553862,E9468AA1B4AEB1GS,1
2196,1049.44,1.0,International XS,F0615AA3F9EFE0GS,223818962,internationalxs,223818962,F0615AA3F9EFE0GS,1


In [161]:
DATA_CONVERSIONS_EXCHANGE_RETURNS.groupby('ind_exchangereturn').size()

ind_exchangereturn
0    8099
1     479
dtype: int64

In [162]:
DATA_CONVERSIONS_EXCHANGE_RETURNS.drop(columns=['Order nr','SKU ID'],inplace=True)

## Step 2 : Append Return Reason if available by matching based on transaction id, sku id and size

In [163]:
DATA_CONVERSIONS_EXCHANGE_RETURNS = DATA_CONVERSIONS_EXCHANGE_RETURNS.merge(
  right = DATA_EXCHANGE_RETURNS,
  how = 'left',
  left_on = ['transaction','sku','size_name'],
  right_on = ['Order nr','SKU ID','original_size']
)

In [164]:
DATA_CONVERSIONS_EXCHANGE_RETURNS = DATA_CONVERSIONS_EXCHANGE_RETURNS.drop(columns=[
    'Order nr',
    'SKU ID',
    'original_size'
])

In [165]:
DATA_CONVERSIONS_EXCHANGE_RETURNS.sort_values('exchanged_size').head()

,price,quantity,size,sku,transaction,size_name,ind_exchangereturn,Reason Name,exchanged_size
30,438,1.0,International L,4F639AA908FCCAGS,299595962,internationall,1,too_large,<na><na>
6146,0,1.0,International M,ED28EAADEF2243GS,292799862,internationalm,1,too_large,<na><na>
6066,1959.2,1.0,International L,EF930AA374ED11GS,231936862,internationall,1,too_large,<na><na>
6057,831.36,1.0,EU 38,5E3E4SH1951D33GS,223128862,eu38,1,too_small,<na><na>
6033,898.5,1.0,International L,9E0D0AAD9B0E23GS,235329862,internationall,1,too_large,<na><na>


In [166]:
DATA_CONVERSIONS_EXCHANGE_RETURNS['Reason Name'].fillna('<No size match>',inplace=True)

In [167]:
DATA_CONVERSIONS_EXCHANGE_RETURNS.groupby(['ind_exchangereturn','Reason Name']).size().reset_index()

,ind_exchangereturn,Reason Name,0
0,0,<No size match>,8099
1,1,<No size match>,86
2,1,too_large,222
3,1,too_small,171


86 out of the 479 product-transaction records that had product exchanges/returns have no size match 

## Examine unmatched size names

In [186]:
unmatched_purchased_sizenames = set(DATA_CONVERSIONS_EXCHANGE_RETURNS[
    (DATA_CONVERSIONS_EXCHANGE_RETURNS.ind_exchangereturn==1) &
    (DATA_CONVERSIONS_EXCHANGE_RETURNS['Reason Name']=='<No size match>')
]['size_name'])